In [9]:
from skimage.segmentation import clear_border
from local_utils import interval_mapping
import matplotlib.pyplot as plt
# import pytesseract
import numpy as np
import imutils
import cv2
import easyocr
import matplotlib.gridspec as gridspec

In [ ]:
img = cv2.imread('./dataset/plate5.jpeg')

img = cv2.resize(img, (620,480) )
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #convert to grey scale
gray = cv2.bilateralFilter(gray, 13, 15, 15)
edged = cv2.Canny(gray, 30, 200) #Perform Edge detection

contours = cv2.findContours(edged.copy(),cv2.RETR_TREE,
                                            cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(contours)
contours = sorted(contours,key=cv2.contourArea, reverse = True)[:10]
screenCnt = np.zeros((4, 1, 2))

# image_copy = img.copy()
# cv2.drawContours(image=image_copy, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)

for c in contours:
    
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.018 * peri, True)

    if len(approx) == 4:
        screenCnt = approx
        break

# if screenCnt.all() == 0:
#     detected = 0
#     print ("No contour detected")
# else:
#     cv2.drawContours(img, [screenCnt], -1, (0, 0, 255), 3)

mask = np.zeros(gray.shape,np.uint8)
new_image = cv2.drawContours(mask, [screenCnt], 0, 255, -1)
new_image = cv2.bitwise_and(img,img,mask=mask)

(x, y) = np.where(mask == 255)
(topx, topy) = (np.min(x), np.min(y))
(bottomx, bottomy) = (np.max(x), np.max(y))
Cropped = gray[topx:bottomx+1, topy:bottomy+1]

cropped_text = interval_mapping(Cropped, 0.0, 1.0, 0, 255)
reader = easyocr.Reader(['en'])
result = reader.readtext(np.uint8(cropped_text))
print(result)

# text = pytesseract.image_to_string(Cropped, config='--psm 11')
# print("programming_fever's License Plate Recognition\n")
# print("Detected license plate Number is:",text)
print(img.shape)
plt.imshow(mask)

fig = plt.figure(figsize=(12,6))
grid = gridspec.GridSpec(ncols=2,nrows=1,figure=fig)
fig.add_subplot(grid[0])
plt.axis(False)
plt.imshow(img)
grid = gridspec.GridSpec(ncols=2,nrows=1,figure=fig)
fig.add_subplot(grid[1])
plt.axis(False)
plt.imshow(Cropped)

# cv2.waitKey(0)
# cv2.destroyAllWindows()